# Image Classification with ConvMixer

Author: Team 4
Last Modified: Aug, 8, 2022
Replace the whole ViT MLP structure with ConvMixer manually

# Setup

In [1]:
from tensorflow.keras import layers
from tensorflow import keras

import matplotlib.pyplot as plt
import tensorflow_addons as tfa
import tensorflow as tf
import numpy as np

## Prepare the data

In [2]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
val_split = 0.1
val_indices = int(len(x_train) * val_split)
x_val, y_val = x_train[:val_indices], y_train[:val_indices]
x_train, y_train = x_train[val_indices:], y_train[val_indices:]

print(f"Training data: {len(x_train)}")
print(f"Validation data: {len(x_val)}")
print(f"Test data: {len(x_test)}")

Training data: 45000
Validation data: 5000
Test data: 10000


## Configure the hyperparameters

In [3]:
learning_rate = 0.001
weight_decay = 0.0001
batch_size = 128
num_epochs = 20
image_size = 32

# Prepare Dataset objects

In [4]:
auto = tf.data.AUTOTUNE
data_augmentation = keras.Sequential(
    [layers.RandomCrop(image_size, image_size), layers.RandomFlip("horizontal"),],
    name="data_augmentation",
)


def make_datasets(images, labels, is_train=False):
    dataset = tf.data.Dataset.from_tensor_slices((images, labels))
    if is_train:
        dataset = dataset.shuffle(batch_size * 10)
    dataset = dataset.batch(batch_size)
    if is_train:
        dataset = dataset.map(
            lambda x, y: (data_augmentation(x), y), num_parallel_calls=auto
        )
    return dataset.prefetch(auto)

train_dataset = make_datasets(x_train, y_train, is_train=True)
val_dataset = make_datasets(x_val, y_val)
test_dataset = make_datasets(x_test, y_test)

# Construct the ConvMixer Model

In [5]:
# model parameters
patch_size = 2
class_num = 10
depth = 8
filter_num = 256
kernel_size = 10

In [6]:
def convMixer_Model():
    # Rescale the input image
    inputs = keras.Input((image_size, image_size, 3))
    rescaled = layers.Rescaling(scale=1.0 / 255)(inputs)
    
    # Patch Embedding
    patch_embed = layers.Conv2D(filter_num, kernel_size=patch_size, strides=patch_size)(rescaled)
    # Gelu
    gelu_out = layers.Activation("gelu")(patch_embed)
    # BatchNorm
    bn_out = layers.BatchNormalization()(gelu_out)

    # ConvMixer blocks with depth iterations
    for _ in range(depth):
        # Depthwise convolution
        temp = bn_out
        dw_conv = layers.DepthwiseConv2D(kernel_size=kernel_size, padding="same")(bn_out)
        # Gelu
        gelu_out = layers.Activation("gelu")(dw_conv)
        # BatchNorm
        bn_out = layers.BatchNormalization()(gelu_out)
        # layer addition
        layer_add = layers.Add()([bn_out, temp])

        # Pointwise convolution
        pw_conv = layers.Conv2D(filter_num, kernel_size=1)(layer_add)
        # Gelu
        gelu_out = layers.Activation("gelu")(pw_conv)
        # BatchNorm
        bn_out = layers.BatchNormalization()(gelu_out)

    # Classification
    globalAvgPool_out = layers.GlobalAvgPool2D()(bn_out)
    do = tf.nn.dropout(globalAvgPool_out, 0.3)
    outputs = layers.Dense(class_num, activation="softmax")(do)

    return keras.Model(inputs, outputs)

# Compile, train, and evaluate the mode

In [7]:
def run_experiment(model):
    optimizer = tfa.optimizers.AdamW(
        learning_rate=learning_rate, weight_decay=weight_decay
    )

    model.compile(
        optimizer=optimizer,
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"],
    )

    checkpoint_filepath = "/tmp/checkpoint"
    checkpoint_callback = keras.callbacks.ModelCheckpoint(
        checkpoint_filepath,
        monitor="val_accuracy",
        save_best_only=True,
        save_weights_only=True,
    )

    history = model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=num_epochs,
        callbacks=[checkpoint_callback],
    )

    model.load_weights(checkpoint_filepath)
    _, accuracy = model.evaluate(test_dataset)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    return history, model

In [8]:
cm_model = convMixer_Model()
history, conv_mixer_model = run_experiment(cm_model)

Epoch 1/20
352/352 [==============================] - 66s 176ms/step - loss: 1.4712 - accuracy: 0.4685 - val_loss: 2.9218 - val_accuracy: 0.1470
Epoch 2/20
352/352 [==============================] - 61s 174ms/step - loss: 0.9918 - accuracy: 0.6505 - val_loss: 0.9662 - val_accuracy: 0.6640
Epoch 3/20
352/352 [==============================] - 61s 175ms/step - loss: 0.7777 - accuracy: 0.7290 - val_loss: 0.7532 - val_accuracy: 0.7416
Epoch 4/20
352/352 [==============================] - 61s 174ms/step - loss: 0.6475 - accuracy: 0.7759 - val_loss: 0.7375 - val_accuracy: 0.7428
Epoch 5/20
352/352 [==============================] - 62s 175ms/step - loss: 0.5529 - accuracy: 0.8078 - val_loss: 0.6474 - val_accuracy: 0.7718
Epoch 6/20
352/352 [==============================] - 61s 174ms/step - loss: 0.4763 - accuracy: 0.8360 - val_loss: 0.5956 - val_accuracy: 0.7910
Epoch 7/20
352/352 [==============================] - 61s 174ms/step - loss: 0.4141 - accuracy: 0.8560 - val_loss: 0.6350 - val_ac